In [1]:
import os 
import numpy as np 
import sys
import os.path as osp
root_path = osp.dirname(os.getcwd())

sys.path.append(root_path)
sys.path.append(osp.join(root_path,'my_script'))

os.chdir(root_path)
os.getcwd()


'/data/xusc/exp/butd_detr'

In [2]:

# os.chdir("/home/DISCOVER_summer2022/xusc/exp/butd_detr")

import glob 

# from src.labeled_arkitscenes_dataset import ARKitSceneDataset
from loguru import logger
from my_script.pc_utils import *
from my_script.utils import *


from src.joint_det_dataset import points2box

def save_(data,scene_name,save_root,has_color= True,flag = "debug"):
      #* for teacher or student 
      
    make_dirs(osp.join(save_root,scene_name))

    if has_color:
        write_pc_as_ply(
                    data['point_clouds'],
                    os.path.join(save_root,scene_name, '%s_gt_%s.ply'%(scene_name,flag))
                )
    else :
        write_ply(
            data['point_clouds'],
            os.path.join(save_root,scene_name, '%s_gt_%s.ply'%(scene_name,flag))
        )

    #* open3d  format bounding box 
    np.savetxt(os.path.join(save_root, scene_name,'%s_box_%s.txt'%(scene_name,flag)),
            data['all_bboxes'][data['all_bbox_label_mask']],fmt='%s')

    




def save_teacher(data,scene_name,save_root,has_color= True,flag = "debug"):
      #* for teacher or student 
      
    make_dirs(osp.join(save_root,scene_name))

    if has_color:
        write_pc_as_ply(
                    data['pc_before_aug'],
                    os.path.join(save_root,scene_name, '%s_gt_%s.ply'%(scene_name,flag))
                )
    else :
        write_ply(
            data['pc_before_aug'],
            os.path.join(save_root,scene_name, '%s_gt_%s.ply'%(scene_name,flag))
        )

        

    #* open3d  format bounding box 
    np.savetxt(os.path.join(save_root, scene_name,'%s_box_%s.txt'%(scene_name,flag)),
            data['teacher_box'][data['all_bbox_label_mask']],fmt='%s')


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## 2.  检查ARKitScene datasets  getitem 的数据格式

In [3]:

from my_script.utils import save_for_vis


fcaf3d_scannet_path = "datasets/fcaf3d_scannet"
all_res = sorted(glob.glob(fcaf3d_scannet_path+"/*.bin"))
debug_path = "logs/debug"
if not osp.exists(debug_path):
    os.makedirs(debug_path)
    
class_ids = []
for d in all_res:
    scene_name = d.split('/')[-1].split('.')[-2]
    
    res = np.fromfile(d,dtype=np.float32).reshape(-1,8)
    class_ids = res[:,-1]
    box =  res[:,:6]
    print(f"{d} ,  length : {box.shape}")
    pc = np.load(osp.join("datasets/scannet_pc",f"{scene_name}_pc.npy"))
    print(f"pc shape : {pc.shape}")
    save_for_vis(box,pc,debug_path,scene_name)
    break


datasets/fcaf3d_scannet/scene0000_00.bin ,  length : (30, 6)
pc shape : (50000, 6)


In [96]:



for idx in range(res.shape[1]):
    
    # print(res[:,idx],'\n\n',res[:,idx].astype(np.int64),"\n\n")
    print(f'dimension: {idx} : ',res[:,idx].astype(np.int64),"\n\n")
    


dimension: 0 :  [   0  -50   10   45    1    0   -2    0  127    9  -47 -125  -41  -28
    0] 


dimension: 1 :  [ 0  0  0 14  0 12  0  0  0  0  0  0  0  0  0] 


dimension: 2 :  [0 0 0 7 0 0 0 1 0 0 0 0 0 0 0] 


dimension: 3 :  [       0        0        0        0        0       32      512     2048
     8192   262144   524288  1048576  8388608 50331648 50331648] 


dimension: 4 :  [ -22    7    0    8   -7    3    3    0 -124    3    0    0   -3    0
   -3] 


dimension: 5 :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 


dimension: 6 :  [1 5 4 0 0 0 0 1 0 0 2 0 0 0 0] 


dimension: 7 :  [       0        0        0        0        0      512      512     2048
     8192   262144  1048576  2097152 16777216 50331648 50331648] 




In [2]:
point_num = 100000
arkit = ARKitSceneDataset( num_points=point_num,augment=True,data_root='datasets/arkitscenes',debug=True)
demo = arkit.__getitem__(1)
demo_attr  = get_attr(demo)
print_attr_shape(demo)

2022-10-22 23:43:15.276 | INFO     | src.labeled_arkitscenes_dataset:__init__:214 - split train : 4361 loaded
2022-10-22 23:43:16.518 | INFO     | src.labeled_arkitscenes_dataset:__init__:214 - split valid : 488 loaded
2022-10-22 23:43:16.519 | INFO     | src.labeled_arkitscenes_dataset:__init__:216 -  total length  : 4849 loaded
2022-10-22 23:43:16.609 | INFO     | src.labeled_arkitscenes_dataset:__init__:222 - ARKitSceneDataset : 4849 sample loaded, scene_name number : 4849 
2022-10-22 23:43:17.677 | INFO     | my_script.utils:print_attr_shape:313 - box_label_mask : (132,)
2022-10-22 23:43:17.687 | INFO     | my_script.utils:print_attr_shape:313 - center_label : (132, 3)
2022-10-22 23:43:17.688 | INFO     | my_script.utils:print_attr_shape:313 - sem_cls_label : (132,)
2022-10-22 23:43:17.689 | INFO     | my_script.utils:print_attr_shape:313 - size_gts : (132, 3)
2022-10-22 23:43:17.690 | INFO     | my_script.utils:print_attr_shape:295 - scan_ids : 40753686
2022-10-22 23:43:17.690 | I

# 检查code

In [4]:
from tqdm import tqdm
tgt_num = []
for idx in tqdm(range(len(arkit))):
    example =  arkit.__getitem__(idx)
    target_num = example['box_label_mask'].astype(np.int64).sum()
    tgt_num.append(target_num)
    # logger.info(f"target number == {target_num}")
    

100%|██████████| 4849/4849 [12:16<00:00,  6.59it/s]


# save for vis test 

In [3]:


debug_path = "logs/debug"
make_dirs(debug_path)
for  idx, data in enumerate(arkit):
    save_(data,data['scan_ids'],debug_path,flag='student')
    save_teacher(data,data['scan_ids'],debug_path,flag='teacher')
    logger.info(f"data['scan_ids']:{data['scan_ids']}")
    if idx ==5 :
        break


2022-10-22 23:43:33.702 | INFO     | __main__:<module>:6 - data['scan_ids']:40753679
2022-10-22 23:43:48.185 | INFO     | __main__:<module>:6 - data['scan_ids']:40753686
2022-10-22 23:44:02.493 | INFO     | __main__:<module>:6 - data['scan_ids']:40776203
2022-10-22 23:44:15.281 | INFO     | __main__:<module>:6 - data['scan_ids']:40776204
2022-10-22 23:44:26.325 | INFO     | __main__:<module>:6 - data['scan_ids']:40777060
2022-10-22 23:44:39.491 | INFO     | __main__:<module>:6 - data['scan_ids']:40777063


In [3]:

debug_path = "logs/debug"
make_dirs(debug_path)
for  idx, data in enumerate(arkit):
    save_(data,data['scan_ids'],debug_path)
    logger.info(f"data['scan_ids']:{data['scan_ids']}")
    if idx ==10 :
        break


2022-10-22 22:28:51.186 | INFO     | src.labeled_arkitscenes_dataset:downsample_ceil:348 - before delete ceil :(104513, 6)
2022-10-22 22:28:51.191 | INFO     | src.labeled_arkitscenes_dataset:downsample_ceil:354 - after  delete ceil :(83610, 6)
2022-10-22 22:28:54.066 | INFO     | __main__:<module>:5 - data['scan_ids']:40753679
2022-10-22 22:28:54.070 | INFO     | src.labeled_arkitscenes_dataset:downsample_ceil:348 - before delete ceil :(97751, 6)
2022-10-22 22:28:54.075 | INFO     | src.labeled_arkitscenes_dataset:downsample_ceil:354 - after  delete ceil :(78200, 6)
2022-10-22 22:28:56.894 | INFO     | __main__:<module>:5 - data['scan_ids']:40753686
2022-10-22 22:29:00.391 | INFO     | src.labeled_arkitscenes_dataset:downsample_ceil:348 - before delete ceil :(193440, 6)
2022-10-22 22:29:04.767 | INFO     | src.labeled_arkitscenes_dataset:downsample_ceil:354 - after  delete ceil :(154752, 6)
2022-10-22 22:29:07.612 | INFO     | __main__:<module>:5 - data['scan_ids']:40776203
2022-10-22

In [5]:

debug_path = "logs/debug"
make_dirs(debug_path)
for  idx, data in enumerate(arkit):
    # if data['scan_ids']=="40777073":
    #     print_attr_shape(data)
    #     break

    logger.info(data['augmentations']['theta_z'])
    logger.info(data['augmentations']['theta_x'])
    logger.info(data['augmentations']['theta_y'])
    if idx ==100:
        break


2022-10-22 22:30:25.047 | INFO     | src.labeled_arkitscenes_dataset:downsample_ceil:348 - before delete ceil :(104513, 6)
2022-10-22 22:30:25.053 | INFO     | src.labeled_arkitscenes_dataset:downsample_ceil:354 - after  delete ceil :(83610, 6)
2022-10-22 22:30:25.155 | INFO     | __main__:<module>:8 - -2.6792118436541603
2022-10-22 22:30:25.162 | INFO     | __main__:<module>:9 - 0.6703702679309548
2022-10-22 22:30:25.163 | INFO     | __main__:<module>:10 - -0.2947173894682553
2022-10-22 22:30:25.166 | INFO     | src.labeled_arkitscenes_dataset:downsample_ceil:348 - before delete ceil :(97751, 6)
2022-10-22 22:30:25.171 | INFO     | src.labeled_arkitscenes_dataset:downsample_ceil:354 - after  delete ceil :(78200, 6)
2022-10-22 22:30:25.243 | INFO     | __main__:<module>:8 - 0.22090562641420397
2022-10-22 22:30:25.248 | INFO     | __main__:<module>:9 - -0.158134767323711
2022-10-22 22:30:25.248 | INFO     | __main__:<module>:10 - -1.3343644148258427
2022-10-22 22:30:25.252 | INFO     | 

KeyboardInterrupt: 

2022-10-20 14:35:27.498 | INFO     | __main__:<module>:4 - demo['scan_ids']:40753679
